In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

#import pandas, xgboost, numpy, textblob, string
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers
import gensim
import umap
import umap.plot
from yellowbrick.text import TSNEVisualizer
import re, string,os
from glob import glob as gb
import pandas as pd
from collections import Counter
from tqdm import tqdm
from datetime import datetime, timedelta, date
from collections import OrderedDict
import subprocess
import fuzzywuzzy
from fuzzywuzzy import process,fuzz
from utils.functions import *
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from nltk.corpus import stopwords
from classification import *

# First test: https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/
# Word2vec: http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/ 

In [2]:
## Import annotated data
annotations = pd.read_csv('~/Documents/GitHub/CrisisBureaucracy/data/classifier/annotated-arguments-bureaucracy.csv')
annotations = annotations[["id","label","text"]]
annotations['metadata'] = ''

refdata = pd.read_csv('~/Documents/GitHub/CrisisBureaucracy/data/classifier/training_data_full.csv',sep='\t')
refdata['id-ann'] = [x + 594 for x in refdata.index]

for c,i in enumerate(annotations['id']):
    annotations['metadata'][c] = str(refdata[refdata['id-ann'] == i].reset_index(drop=True)['id'][0])


stops = stopwords.words('english') + ["hon","member","right","friend","mr",'hon.','make','say','great']
annotations['text'] = utils.preprocess_(annotations['text'])
annotations = annotations.drop('id',axis=1).reset_index(drop=True)

labels = {1:"neutral",2:"inefficient",3:"powerful/large",4:"centralization",5:"freedom",6:"expensive",7:"anti-democratic"}
annotations['label'] = annotations['label'].astype(str)

#annotations = annotations[(annotations['label'] != '1') & (annotations['label'] != '3')].reset_index(drop=True)

# model = gensim.models.KeyedVectors.load_word2vec_format('/media/ruben/Elements/GoogleNews-vectors-negative300-SLIM.bin/GoogleNews-vectors-negative300-SLIM.bin',binary=True)
# w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [19]:
# Inspect Category TF-IDF terms
tfidfo, docterms = tfidf.get_docterms(annotations,"text")
tt = tfidf.get_topterms(tfidfo,docterms,annotations,'label')
#tt.head(10)

In [33]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(annotations['text'], annotations['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(annotations['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [5]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [35]:
# Naive Bayes on Ngram Level TF IDF Vectors
#accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print("NB, N-Gram Vectors: ", accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.45023696682464454


In [11]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(annotations['text'], annotations['label'],train_size=0.5)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfEmbeddingVectorizer(word2vec=w2v) #TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(annotations['text'],annotations['label'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

#TfidfEmbeddingVectorizer()